In [1]:
import tensorflow as tf
# tf.test.is_gpu_available(
#     cuda_only=False, min_cuda_compute_capability=None
# )
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import csv
import shutil

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.layers import Input, Dense, Lambda, Dropout, Conv1D, MaxPool1D, Flatten, UpSampling1D, AveragePooling1D#, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.python.keras import utils
from tensorflow.keras.callbacks import TensorBoard

In [4]:
path_to_dataset = '../dataset/N_DATA'

In [5]:
def load_dataset():
    x, y = [], []
    directory = os.fsencode(path_to_dataset)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        pos = filename.find('_')
        label = filename[pos+1:-4]
        data = []
        with open(path_to_dataset+'/'+filename, newline='') as f:
            reader = csv.reader(f, delimiter=';')
            data = [[float(y) for y in x] for x in list(reader)]
        data = np.array(data).T.tolist()
        data.pop(0)
        data.pop(-1)
        x.append(np.array(data).flatten())
        y.append(int(label))
    return np.array(x), np.array(y)

In [6]:
'''
Удалим первые нули чтобы форма была (400, 7) для сохранения формы после пуллинга
'''
x, y = load_dataset()
x = np.array([point.reshape(401, 7) for point in x])
x = np.array([point[1:] for point in x])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(411, 400, 7) (137, 400, 7) (411,) (137,)


In [15]:
input_layer = Input(shape=(400, 7))

def conv(kernels, kernel_size, pool, latent_dim=64, activation='relu'):
    
    encoded = Conv1D(kernels, kernel_size, activation = 'relu', padding = 'same')(input_layer)
    encoded = MaxPool1D(pool, padding='same')(encoded)
    
    encoded = Conv1D(kernels, kernel_size, activation = 'relu', padding = 'same')(encoded)
    encoded = MaxPool1D(pool, padding='same')(encoded)
    
#     encoded = Flatten()(encoded)
#     encoded = Dense(latent_dim)(encoded)

    decoded = Conv1D(kernels, kernel_size, activation = 'relu', padding = 'same')(encoded)
    decoded = UpSampling1D(pool)(decoded)
    
    decoded = Conv1D(kernels, kernel_size, activation = 'relu', padding = 'same')(decoded)
    decoded = UpSampling1D(pool)(decoded)
    
    decoded = Conv1D(7, kernel_size, activation = 'sigmoid', padding = 'same')(decoded)

    autoencoder = Model(input_layer, decoded)
    return autoencoder

In [9]:
pool = 2
kernels = 17
kernel_size = 7
losses = []

for i in range(5):
    autoencoder = conv(kernels, kernel_size, pool)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    history = autoencoder.fit(x_train, x_train,
                                  epochs=160,
                                  batch_size=16,
                                  shuffle=True,
                                  verbose=0,
                                  validation_data=(x_test, x_test))
    current_loss = history.history['val_loss'][-1]
    losses.append(current_loss)
    print(f'iter = {i+1} loss = {current_loss}')

print(f'Average loss = {np.sum(losses) / 5}')

iter = 1 loss = 0.22165244817733765
iter = 2 loss = 0.22199948132038116
iter = 3 loss = 0.2219565212726593
iter = 4 loss = 0.22168411314487457
iter = 5 loss = 0.22172857820987701
Average loss = 0.22180422842502595


In [10]:
'''
Стандартная загрузка
'''
x, y = load_dataset()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(411, 2807) (137, 2807) (411,) (137,)


In [18]:
input_layer = Input(shape=(2807,))

def basic(n_layers, latent_dim, k=2, activation='relu'):
    encoded = Dense(n_layers*latent_dim, activation=activation)(input_layer)
    for current_dim in range(n_layers - 1, 0, -1):
        encoded = Dense(current_dim*latent_dim, activation=activation)(encoded)
    
    if n_layers > 1:
        decoded = Dense(2*latent_dim, activation=activation)(encoded)
        for current_dim in range(3, n_layers + 1):
            decoded = Dense(current_dim*latent_dim, activation=activation)(decoded)
        decoded = Dense(2807, activation='sigmoid')(decoded)
    else:
         decoded = Dense(2807, activation='sigmoid')(encoded)
    
    autoencoder = Model(input_layer, decoded)
    
    return autoencoder

In [12]:
layers = 3
latent_dim = 64
losses = []

for i in range(5):
    autoencoder = basic(layers, latent_dim, activation='elu')
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    history = autoencoder.fit(x_train, x_train,
                                  epochs=160,
                                  batch_size=16,
                                  shuffle=True,
                                  verbose=0,
                                  validation_data=(x_test, x_test))
    current_loss = history.history['val_loss'][-1]
    losses.append(current_loss)
    print(f'iter = {i+1} loss = {current_loss}')

print(f'Average loss = {np.sum(losses) / 5}')

iter = 1 loss = 0.22289322316646576
iter = 2 loss = 0.2225545048713684
iter = 3 loss = 0.22272862493991852
iter = 4 loss = 0.22264377772808075
iter = 5 loss = 0.22258934378623962
Average loss = 0.2226818948984146


In [13]:
layers = 2
latent_dim = 128
losses = []

for i in range(5):
    autoencoder = basic(layers, latent_dim, activation='elu')
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    history = autoencoder.fit(x_train, x_train,
                                  epochs=160,
                                  batch_size=16,
                                  shuffle=True,
                                  verbose=0,
                                  validation_data=(x_test, x_test))
    current_loss = history.history['val_loss'][-1]
    losses.append(current_loss)
    print(f'iter = {i+1} loss = {current_loss}')

print(f'Average loss = {np.sum(losses) / 5}')

iter = 1 loss = 0.22221428155899048
iter = 2 loss = 0.22217793762683868
iter = 3 loss = 0.22226083278656006
iter = 4 loss = 0.2222176343202591
iter = 5 loss = 0.22231797873973846
Average loss = 0.22223773300647737


|            | Число обучаемых параметров | Число выделяемых признаков | Средняя ошибка |
|------------|----------------------------|----------------------------|----------------|
| Сверточная | 7810                       | 1700                       | 0.22180        |
| 3 слоя     | 1146935                    | 64                         | 0.22268        |
| 2 слоя     | 1506167                    | 128                        | 0.22224        |